In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import faiss
import numpy as np


In [3]:
retriever_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the retriever model
retriever_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Load data from a CSV file
df = pd.read_csv("your_file.csv")  # Replace with your file path
passages = df["text"].dropna().tolist()  # Assume the relevant text is in a column named "text"

# Encode the passages
passage_embeddings = retriever_model.encode(passages, convert_to_tensor=True)

print("Encoded passages:", passage_embeddings.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'your_file.csv'

In [ ]:
index = faiss.IndexFlatL2(passage_embeddings.shape[1])
index.add(np.array(passage_embeddings))


In [ ]:
generator_tokenizer = AutoTokenizer.from_pretrained("t5-small")
generator_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


In [ ]:
def retrieve_and_generate(query, top_k=5):
    # Retrieve relevant passages
    query_embedding = retriever_model.encode([query], convert_to_tensor=True)
    _, indices = index.search(np.array(query_embedding), top_k)
    retrieved_passages = [passages[i] for i in indices[0]]
    
    # Combine passages for generation
    input_text = " ".join(retrieved_passages)
    input_ids = generator_tokenizer.encode(query + " " + input_text, return_tensors="pt", truncation=True)

    # Generate response
    outputs = generator_model.generate(input_ids, max_length=200, num_beams=5)
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example query
query = "What are the benefits of solar energy?"
print(retrieve_and_generate(query))
